<a href="https://colab.research.google.com/github/VedeshP/nl2sql_try/blob/main/nl2sql_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### About dataset

Using wikisql salesforce dataset
since dataset is hosted on kaggle - using it directly from datasets library from kaggle

huggingface link

https://huggingface.co/datasets/Salesforce/wikisql

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset("Salesforce/wikisql", trust_remote_code=True)

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

wikisql.py:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 56355
    })
})


In [6]:
print(dataset["train"])

Dataset({
    features: ['phase', 'question', 'table', 'sql'],
    num_rows: 56355
})


In [7]:
print(dataset["train"][0])

{'phase': 1, 'question': 'Tell me what the notes are for South Australia ', 'table': {'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'page_title': '', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-1000181-1', 'section_title': '', 'caption': '', 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'black/white', 'aaa·nna', 'NSW', 'CPX·12A', 'Optional white slimline series'], ['Northern Territory', 'ochre/white', 'Ca·nn·aa', 'NT · OUTBACK AUSTRALIA', 'CB·06·ZZ', 'New series began in June 2011'], ['Queensland', 'maroon/white', 'nnn·aaa', 'QUEENSLAND · SUNSHINE STATE', '999·TLG', 'Slogan embossed on plate'], ['South Australia', 'black/white', 'Snnn·aaa', 'SOUTH AUS

In [8]:
for i in range(5):
    print("Question:", dataset['train'][i]['question'])
    print("SQL Query:", dataset['train'][i]['sql']['human_readable'])
    print("Table Schema:", dataset['train'][i]['table']['header'])
    print("-" * 50)

Question: Tell me what the notes are for South Australia 
SQL Query: SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA
Table Schema: ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
--------------------------------------------------
Question: What is the current series where the new series began in June 2011?
SQL Query: SELECT Current series FROM table WHERE Notes = New series began in June 2011
Table Schema: ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
--------------------------------------------------
Question: What is the format for South Australia?
SQL Query: SELECT Format FROM table WHERE State/territory = South Australia
Table Schema: ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
--------------------------------------------------
Question: Name the background colour for the Australian Capital Territory
SQL 

In [9]:
print(dataset["train"].features)

{'phase': Value(dtype='int32', id=None), 'question': Value(dtype='string', id=None), 'table': {'header': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'page_title': Value(dtype='string', id=None), 'page_id': Value(dtype='string', id=None), 'types': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'id': Value(dtype='string', id=None), 'section_title': Value(dtype='string', id=None), 'caption': Value(dtype='string', id=None), 'rows': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None), 'name': Value(dtype='string', id=None)}, 'sql': {'human_readable': Value(dtype='string', id=None), 'sel': Value(dtype='int32', id=None), 'agg': Value(dtype='int32', id=None), 'conds': Sequence(feature={'column_index': Value(dtype='int32', id=None), 'operator_index': Value(dtype='int32', id=None), 'condition': Value(dtype='string', id=None)}, length=-1, id=None)}}


In [10]:
table_schema_example = dataset["train"][0]['table']['header']
print(table_schema_example)

['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']


In [11]:
example = dataset["train"][0]

In [12]:
def format_example(example):
    # Extract fields
    question = example['question']
    sql_query = example['sql']['human_readable']
    table_schema = ', '.join(example['table']['header'])  # Convert list to string

    # Format input to include schema
    input_text = f"Question: {question} | Table: {table_schema}"

    return {
        "input_text": input_text,
        "output_text": sql_query
    }


https://huggingface.co/docs/datasets/en/process#map

In [13]:
print(format_example(example))

{'input_text': 'Question: Tell me what the notes are for South Australia  | Table: State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'output_text': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}


In [14]:
formatted_data_ex = format_example(example)
print(formatted_data_ex)

{'input_text': 'Question: Tell me what the notes are for South Australia  | Table: State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'output_text': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}


In [15]:
# formatted_data = dataset.map(lambda x: format_example(x))

formatted_data = dataset.map(format_example, remove_columns=["phase", "table", "sql", "question"])


Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [16]:
print(formatted_data["train"][0])

{'input_text': 'Question: Tell me what the notes are for South Australia  | Table: State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'output_text': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}


In [17]:
print(dataset["train"][0])

{'phase': 1, 'question': 'Tell me what the notes are for South Australia ', 'table': {'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'page_title': '', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-1000181-1', 'section_title': '', 'caption': '', 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'black/white', 'aaa·nna', 'NSW', 'CPX·12A', 'Optional white slimline series'], ['Northern Territory', 'ochre/white', 'Ca·nn·aa', 'NT · OUTBACK AUSTRALIA', 'CB·06·ZZ', 'New series began in June 2011'], ['Queensland', 'maroon/white', 'nnn·aaa', 'QUEENSLAND · SUNSHINE STATE', '999·TLG', 'Slogan embossed on plate'], ['South Australia', 'black/white', 'Snnn·aaa', 'SOUTH AUS

In [18]:
from transformers import T5TokenizerFast

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [19]:
tokenizer = T5TokenizerFast.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [20]:
# Define tokenization function
def tokenize_function(example):
    # Tokenize input (question + table schema)
    input_encodings = tokenizer(
        example["input_text"],
        padding="max_length",  # Ensures all inputs are the same length
        truncation=True,  # Truncates if input is too long
        max_length=512  # Standard max length for Transformer models
    )

    # Tokenize output (SQL query)
    output_encodings = tokenizer(
        example["output_text"],
        padding="max_length",
        truncation=True,
        max_length=128  # SQL queries are usually shorter
    )

    # Return tokenized inputs and labels
    return {
        "input_ids": input_encodings["input_ids"],  # Tokenized input text
        "attention_mask": input_encodings["attention_mask"],  # Indicates non-padding tokens
        "labels": output_encodings["input_ids"],  # Tokenized SQL query
    }


In [21]:
tokenized_dataset = formatted_data.map(tokenize_function, batched=True, remove_columns=["input_text", "output_text"])

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [22]:
print(tokenized_dataset["train"][0])

{'input_ids': [11860, 10, 8779, 140, 125, 8, 3358, 33, 21, 1013, 2051, 1820, 4398, 10, 1015, 87, 17, 21301, 10972, 6, 5027, 87, 1549, 9232, 3243, 6, 12439, 6, 12892, 22031, 6, 12892, 939, 6, 2507, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [23]:
!pip3 install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [24]:
from torch.utils.data import Dataset

class SQLDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}